In [3]:
%matplotlib inline
from pylab import *
import numpy as np
from collections import deque

In [45]:
class Queue:
    """A queue for use in simulating M/M/1/k.
    
    Attributes:
        k (int): Maximum customers allowed in the system.
        departures (list): A sample of departure intervals.
        queue (list): A deque object.
        dropped (int): Number of items dropped because queue was full.
        served (int): Number of items served from queue.
        
    Todo:
        * Clean up after simulation method is done.
    """
    
    def __init__(self, k, mu, departures):
        """Forms a queue.

        Args:
            k (int): Maximum customers allowed in the system.
            mu (float): Rate out of the queue.
            departures (int): Number of departure intervals to generate.
        """
        self.k = k
        self.departures = exponential(1/mu, departures)
        self.queue = deque([], k)
        self.dropped = 0
        self.served = 0
        
    def empty(self):
        """Checks if the queue is empty.
        
        Returns:
            True if empty, False otherwise.
        """
        return len(self.queue) is 0
    
    def is_full(self):
        """Checks if the queue is full.
        
        Returns:
            True if full, False otherwise.
        """
        return len(self.queue) is self.k
    
    def enqueue(self, item):
        """Adds an item to end of the queue.
        
        Args:
            item: An item to add to the queue.
        """
        if self.is_full():
            self.dropped += 1
        else:
            self.queue.append(item)
        
    def dequeue(self):
        """Removes the fist item from the queue."""
        if not self.empty():
            self.served += 1
            return self.queue.popleft()
        return -1
            
    def get_size(self):
        """Get the size of the queue.
        
        Returns:
            An integer for the size of the queue.
        """
        return len(self.queue)

In [140]:
def simulation(lamb, mu, k, phi, samples=6000):
    """Used to run a simulation of an M/M/1/k network.
    
    Args:
        lamb (float): The rate into the entire network.
        mu (float): The rate out of the two queues in the network.
        k (int): Maximum number of customers the two queues can handle.
        phi (float): Probability an arrival goes to the first queue.
        samples (int): Number of packets to sample. Defaults to 6000.
        
    Todo:
        * Sort out problems with delay.
        * Add throughput.
        * Clean up code and add comments where needed.
        * Add pretty charts and graphs for metrics.
    """
    queue1 = Queue(k, mu, samples*2)
    queue2 = Queue(k, mu, samples*2)
    # Counts arrivals to each node.
    queue1_arrivals, queue2_arrivals = 0, 0
    # Count time passed.
    time = 0
    # Indexes for sample space lists.
    i, j, n, m = 0, 0, 0, 0
    # Lists for obtaining average number of packets in queue.
    queue1_size, queue2_size = [], []
    queue1_time, queue2_time = [0], [0]
    # Iterate over entire sample of arrivals.
    while queue1.served < samples and queue2.served < samples:
        # Generate an interarrival time.
        arrivals = exponential(1/lamb)
        # Idle state, ignores output rates.
        if time is 0:
            if random() < phi:
                queue1_arrivals += 1
                queue1.enqueue(0)
            else:
                queue2_arrivals += 1
                queue2.enqueue(0)
            # Increments time by one arrival interval.
            time += arrivals
        else:
            # Dequeues any packets that should have been processed
            # before the next arrival.
            while queue1.departures[i] <= time:
                t = queue1.dequeue() 
                if t is not -1:
                    queue1_time.append(queue1.departures[i] - t)
                # Sums the intervals to compare with time since arrival.
                queue1.departures[i+1] += queue1.departures[i]
                i += 1
                if queue1.served > 1000:
                    queue1_size.append(queue1.get_size())
            while queue2.departures[j] <= time:
                t = queue2.dequeue() 
                if t is not -1:
                    queue2_time.append(queue2.departures[j] - t)
                queue2.departures[j+1] += queue2.departures[j]
                j += 1
                if queue2.served > 1000:
                    queue2_size.append(queue2.get_size())
            # Splits arrivals based on phi probability.
            if random() < phi:
                queue1_arrivals += 1
                queue1.enqueue(time)
            else:
                queue2_arrivals += 1
                queue2.enqueue(time)
            if queue1.served > 1000 or queue2.served > 1000:
                queue1_size.append(queue1.get_size())
                queue2_size.append(queue2.get_size())
            # Increments time by one arrival interval.
            time += arrivals
            
    # Print metrics here.
    # Queue 1.
    e_pb1 = eval_blocking(lamb*phi, mu, k)
    pb1 = queue1.dropped/queue1_arrivals
    e_et1 = eval_delay(lamb*phi, mu, k, e_pb1)
    et1 = average(queue1_time)
    # Queue 2.
    e_pb2 = eval_blocking(lamb*(1-phi), mu, k)
    pb2 = queue2.dropped/queue2_arrivals
    e_et2 = eval_delay(lamb*(1-phi), mu, k, e_pb2)
    et2 = average(queue2_time)
    # Whole system.
    e_pb = phi*e_pb1 + (1-phi)*e_pb2
    pb = (queue1.dropped+queue2.dropped)/(queue1_arrivals + queue2_arrivals)
    e_et = phi*e_et1 + (1-phi)*e_et2
    et = average(queue1_time+queue2_time)
    
    # Whole system.
    print("System:")
    print("\tBlocking probability:\n\t\tExpected: ", e_pb)
    print("\t\tSimulated: ", pb)
    print("\tAverage delay:\n\t\tExpected: ", e_et)
    print("\t\tSimulated: ", et)
    # Queue 1.
    print("\nQueue 1:")
    print("\tBlocking probability:\n\t\tExpected: ", e_pb1)
    print("\t\tSimulated: ", pb1)
    print("\tAverage delay:\n\t\tExpected: ", e_et1)
    print("\t\tSimulated: ", et1)
    # Queue 2.
    print("\nQueue 2:")
    print("\tBlocking probability:\n\t\tExpected: ", e_pb2)
    print("\t\tSimulated: ", pb2)
    print("\tAverage delay:\n\t\tExpected: ", e_et2)
    print("\t\tSimulated: ", et2)
    
    rho = phi*lamb/mu
    en = (rho/(1-rho))-((k+1)*rho**(k+1)/(1-rho**(k+1)))

def eval_blocking(lamb, mu, k):
    """Finds the blocking probability of a queue.
    
    Args:
        lamb: The rate into the queue.
        mu: The rate out of the queue.
        k: Maximum number of customers able to be in the queue.
    """
    rho = lamb/mu
    return rho**k*((1-rho)/(1-rho**(k+1)))

# TODO: Add commenting and sort out problems with delay.
def eval_delay(lamb, mu, k, pb):
    rho = lamb/mu
    en = (rho/(1-rho))-((k+1)*rho**(k+1)/(1-rho**(k+1)))
    return en/(lamb*(1-pb))

In [141]:
simulation(8, 5, 5, 0.6, 6000)

System:
	Blocking probability:
		Expected:  0.10668174420017046
		Simulated:  0.10648030495552732
	Average delay:
		Expected:  0.524315911505353
		Simulated:  0.511109158099

Queue 1:
	Blocking probability:
		Expected:  0.15013154146406452
		Simulated:  0.14531739254198692
	Average delay:
		Expected:  0.5836829818729988
		Simulated:  0.565714577447

Queue 2:
	Blocking probability:
		Expected:  0.04150704830432939
		Simulated:  0.04938271604938271
	Average delay:
		Expected:  0.43526530595388413
		Simulated:  0.438994914809
